In [1]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel

# LangChain OpenAI
from langchain_openai import ChatOpenAI

In [2]:
## .env 설정

load_dotenv()

True

In [4]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel

# LangChain OpenAI
from langchain_openai import ChatOpenAI

def api_document_evaluation_chain():
    llm = ChatOpenAI(model="gpt-4o", temperature=0)

    evaluation_prompt = PromptTemplate.from_template(
        """
    당신은 API 문서 기반 평가 전문가입니다. 주어진 시스템의 답변이 **기준 정답과 얼마나 일치하는지**, **중요한 정보를 얼마나 잘 재현했는지**, 그리고 **세부 사항을 얼마나 잘 반영했는지** 평가해야 합니다.

    평가 기준:
    1. **정확도 (Accuracy)**: 시스템의 답변이 **기준 정답**과 얼마나 일치하는지 평가하세요. 정확도가 높을수록 점수가 높습니다.
    2. **재현율 (Recall)**: 시스템의 답변이 **기준 정답에서 제공한 중요한 정보**(예: 매개변수, 반환값 등)를 얼마나 잘 재현했는지 평가하세요. 중요한 정보가 빠지지 않으면 점수가 높습니다.
    3. **구체성 (Specificity)**: 시스템의 답변이 **기준 정답에서 제공한 세부 사항**(예: 매개변수 설명, 예시 코드 등)을 얼마나 구체적으로 반영했는지 평가하세요. 답변이 상세할수록 점수가 높습니다.

    ## 예시 1:
    **질문**: 구글 지도 API는 어떻게 작동하나요?
    **기준 정답 (API 문서)**: "구글 지도 API는 지도 삽입, 지리 위치 기반 기능, 길찾기 서비스 등을 제공합니다. 이를 통해 사용자는 웹 애플리케이션에 위치 기반 서비스를 통합할 수 있습니다. 예를 들어, 지오코딩, 역 지오코딩 및 거리 계산 기능을 제공합니다."
    **시스템 답변 (Perplexity)**: "구글 지도 API는 지도 삽입과 위치 기반 기능을 제공하며, 길찾기 서비스도 지원합니다."

    ### 평가:
    - **정확도 (Accuracy)**: 점수는 0에서 1 사이로 평가하세요. 1은 완전 일치, 0은 전혀 일치하지 않음을 의미합니다.
    - **재현율 (Recall)**: 점수는 0에서 1 사이로 평가하세요. 1은 모든 핵심 정보를 재현, 0은 중요한 정보를 놓친 경우입니다.
    - **구체성 (Specificity)**: 점수는 0에서 1 사이로 평가하세요. 1은 매우 구체적이고 명확한 설명, 0은 불명확한 설명입니다.

    ## 예시 2:
    **질문**: `getDistance` 메서드의 입력 매개변수는 무엇인가요?
    **기준 정답 (API 문서)**: "`getDistance` 메서드는 두 개의 매개변수를 받습니다: `origin` (경로의 시작점, 문자열)과 `destination` (경로의 종료점, 문자열)입니다. 이 메서드는 시작점과 종료점 간의 거리를 계산하여 반환합니다."
    **시스템 답변 (Perplexity)**: "`getDistance` 메서드는 `origin`과 `destination` 매개변수를 사용하여 거리를 계산합니다."

    ### 평가:
    - **정확도 (Accuracy)**: 점수는 0에서 1 사이로 평가하세요.
    - **재현율 (Recall)**: 점수는 0에서 1 사이로 평가하세요.
    - **구체성 (Specificity)**: 점수는 0에서 1 사이로 평가하세요.

    ## 사용자 질문 : {question}
    ## 기준 정답 : {answer}
    ## 시스템 답변 : {system_answer}

    시스템의 답변을 기반으로 정확도, 재현율, 구체성을 평가한 후, 각 항목에 대한 점수를 **0에서 1 사이**로 계산하여 제공합니다.
    """
    )


    evaluation_chain = evaluation_prompt | llm | StrOutputParser

    return evaluation_chain

In [5]:
## 이미 생성된 합성 데이터셋 생성
import pandas as pd

# 같은 폴더(현재 작업 디렉토리)에 저장했다면
dataset_df = pd.read_csv("./dataset.csv")
print(dataset_df.shape)
display(dataset_df)

(20, 3)


,user_input,reference_contexts,reference
0,Analytics Hub 게시자와 목록 관리자 역할의 차이점은 무엇인가요?,사용자가 다음 작업을 수행할 수 있게 해주는 Analytics Hub 게시자 역할(...,"Analytics Hub 게시자 역할은 목록을 생성, 업데이트, 삭제하고 목록에 I..."
1,BigQuery Studio 저장된 쿼리에 공개 액세스 권한을 방지하려면 어떻게 해...,BigQuery Studio 저장된 쿼리에 공개 액세스 권한이 부여되지 않도록 하려...,BigQuery Studio 저장된 쿼리에 공개 액세스 권한을 방지하려면 프로젝트에...
2,calendarId에 primary 키워드를 사용하면 어떤 캘린더에 접근할 수 있나요?,현재 로그인한 사용자의 기본 캘린더에 액세스하려면 'primary' 키워드를 사용하세요.,calendarId에 primary 키워드를 사용하면 현재 로그인한 사용자의 기본 ...
3,Google Drive API v3의 서비스 엔드포인트가 궁금해,"이 서비스에는 다음 서비스 엔드포인트가 포함되고, 그 아래의 모든 URI가 이 서비...",Google Drive API v3의 서비스 엔드포인트는 https://www.go...
4,files.create 메서드의 최대 파일 크기와 허용되는 미디어 MIME 유형은 ...,"최대 파일 크기: 5,120GB\n허용되는 미디어 MIME 유형: */*\n(리터럴...","files.create 메서드의 최대 파일 크기는 5,120GB입니다. 허용되는 미..."
5,Firebase scrypt 해시 비밀번호를 사용한 사용자 가져오는 방법은?,기본적으로 Firebase는 scrypt 해싱 알고리즘의 수정된 Firebase 버...,"Firebase scrypt 해시 비밀번호를 사용한 사용자를 가져오려면, 먼저 Fi..."
6,커스텀 인증 시스템으로 Firebase에 인증하려면 어떻게 해야 하나요?,Firebase 인증에 맞춤 인증 시스템을 통합할 수 있습니다. 사용자가 정상적으로...,"커스텀 인증 시스템으로 Firebase에 인증하려면, 먼저 인증 서버에서 사용자의 ..."
7,"Firestore에서 맵 필드에 대한 단일 필드 색인 예외를 설정하면, 하위 필드들...",맵 필드에 단일 필드 색인 예외를 만들면 맵의 하위 필드에서 해당 설정을 상속합니다...,"네, Firestore에서 맵 필드에 단일 필드 색인 예외를 만들면 맵의 하위 필드..."
8,Firebase의 updateTime 필드에 사용되는 시간 형식이 뭐야?,"Uses RFC 3339, where generated output will alw...",Firebase의 updateTime 필드에는 RFC 3339 시간 형식이 사용됩니...
9,Firefox 58 환경에서 Gmail의 AMP 렌더링 기능이 호환되나요?,브라우저 최소 버전\nChrome 69\nEdge 지원되지 않음\nFirefox 5...,"네, Firefox 58 환경에서는 Gmail의 AMP 렌더링 기능이 호환됩니다. ..."


In [7]:
## chain 생성

eval_chain = api_document_evaluation_chain()

In [6]:
dataset_df['user_input'][0]

'Analytics Hub 게시자와 목록 관리자 역할의 차이점은 무엇인가요?'